<a href="https://colab.research.google.com/github/FaizFP/Dicoding--Analystic-Sentiments/blob/main/Scraping_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install google-play-scraper
!pip install pandarallel

import pandas as pd
from google_play_scraper import reviews_all,reviews, Sort
!pip install nltk sastrawi
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
from pandarallel import pandarallel




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=9ab905b8265cda854baaede3fec4cb5a2e68ffe5fdc778b6537da9e4f1931079
  Stored in directory: /root/.cache/pip/wheels/46/f9/0d/40c9cd74a7cb8dc8fe57e8d6c3c19e2c730449c0d3f2bf66b5
Successfully built pandarallel
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.6 MB/s eta 0:00:00


In [ ]:
APP_ID = "com.gojek.app"

In [ ]:
# Mengambil 11000 ulasan TERBARU.
# Kita gunakan reviews(), bukan reviews_all(), agar bisa dibatasi.
result, continuation_token = reviews(
    APP_ID,
    lang='id',
    country='id',
    sort=Sort.NEWEST,  # Opsi: mengambil ulasan yang paling baru
    count=11000,       # Batasi jumlah ulasan yang diambil
    filter_score_with=None # Ambil semua skor agar bisa kita labeli nanti
)

# Mengubah hasil scraping menjadi DataFrame pandas
df_reviews = pd.DataFrame(result)

# Menampilkan informasi hasil scrape
print(f"Berhasil mengambil {len(df_reviews)} ulasan.")
print(df_reviews.head())

Berhasil mengambil 11000 ulasan.
                               reviewId         userName  \
0  c650db79-43ec-413a-9c5f-65101db02ca0  Pengguna Google   
1  acb71a96-9f38-4686-91c8-b6aee9662026  Pengguna Google   
2  cb39a538-ea0e-4de8-a8ce-28f8cb972170  Pengguna Google   
3  720a9ac3-75d2-4bec-8a29-933de466cf8f  Pengguna Google   
4  40e74571-4913-49bb-b75d-60045df0e598  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0  aplikasi ini sangat membantu saya untuk order ...      5              0   
1  Drivernya banyak, mau ambil pas hujan juga, ga...      5              0   
2                    Wow uda 10 tah

In [ ]:
df_reviews.head(10)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,c650db79-43ec-413a-9c5f-65101db02ca0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,aplikasi ini sangat membantu saya untuk order ...,5,0,5.33.1,2025-09-30 09:22:15,None,NaT,5.33.1
1,acb71a96-9f38-4686-91c8-b6aee9662026,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Drivernya banyak, mau ambil pas hujan juga, ga...",5,0,5.31.2,2025-09-30 09:16:03,None,NaT,5.31.2
2,cb39a538-ea0e-4de8-a8ce-28f8cb972170,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Wow uda 10 tahin aja 30 09 2025,5,0,5.19.2,2025-09-30 08:51:51,None,NaT,5.19.2
3,720a9ac3-75d2-4bec-8a29-933de466cf8f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,good,5,0,5.33.1,2025-09-30 08:45:09,None,NaT,5.33.1
4,40e74571-4913-49bb-b75d-60045df0e598,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kecewa bgt ga segercep grab padahal lebih seri...,1,0,None,2025-09-30 08:26:44,"Hai Kak Khanza, mohon maaf atas ketidaknyamana...",2025-09-30 16:25:47,None
5,2c703762-827c-46a2-bef4-e8fc530b405e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,👍,5,0,5.33.1,2025-09-30 08:20:12,None,NaT,5.33.1
6,0775a316-b49f-43dc-8405-61631a1a0e1c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus,5,0,5.33.1,2025-09-30 08:19:29,None,NaT,5.33.1
7,874b4a2e-caa4-4619-ae15-7e24ef01f910,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Saya Sangat Kecewa dengan Gojek,dalam Fitur Go...",1,0,5.33.1,2025-09-30 08:13:41,"Hai Kak Hendayani, mohon maaf atas kendala yan...",2025-09-30 16:24:08,5.33.1
8,e42cd916-c869-476b-89c2-0175e6baf7ae,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,gojek mantap,5,0,5.33.1,2025-09-30 08:03:40,None,NaT,5.33.1
9,cd49d245-069b-4a16-80af-bc873cef7210,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sakit hati harus ya resto ya PM,1,0,None,2025-09-30 07:40:27,None,NaT,None


In [ ]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              11000 non-null  object        
 1   userName              11000 non-null  object        
 2   userImage             11000 non-null  object        
 3   content               11000 non-null  object        
 4   score                 11000 non-null  int64         
 5   thumbsUpCount         11000 non-null  int64         
 6   reviewCreatedVersion  8547 non-null   object        
 7   at                    11000 non-null  datetime64[ns]
 8   replyContent          3035 non-null   object        
 9   repliedAt             3035 non-null   datetime64[ns]
 10  appVersion            8547 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 945.4+ KB


In [ ]:
# 2. Inisialisasi pandarallel (ini akan mengaktifkan mode paralel)
# progress_bar=True akan menampilkan bar kemajuan agar Anda bisa memantau prosesnya
pandarallel.initialize(progress_bar=True)

# 3. Terapkan fungsi preprocessing menggunakan .parallel_apply()
# Perhatikan kita menggunakan .parallel_apply() BUKAN .apply()
print("Memulai proses pembersihan teks menggunakan Parallel Processing...")

df_reviews['clean_content'] = df_reviews['content'].parallel_apply(preprocess_text)

print("\nProses pembersihan teks selesai!")

# Tampilkan hasilnya untuk memastikan
print(df_reviews[['content', 'clean_content']].head())

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Memulai proses pembersihan teks menggunakan Parallel Processing...



Proses pembersihan teks selesai!
                                             content  \
0  aplikasi ini sangat membantu saya untuk order ...   
1  Drivernya banyak, mau ambil pas hujan juga, ga...   
2                    Wow uda 10 tahin aja 30 09 2025   
3                                               good   
4  kecewa bgt ga segercep grab padahal lebih seri...   

                                       clean_content  
0  aplikasi bantu order gojek berangkat pulang kerja  
1  drivernya ambil pas hujan ga kaya grab yg maha...  
2                                  wow uda tahin aja  
3                                               good  
4  kecewa bgt ga segercep grab pake gojek kapok b...  


In [ ]:
df_reviews.to_csv('ulasan_gojek_bersih.csv', index=False)

print("Proses berhasil! DataFrame yang sudah bersih telah disimpan ke file 'ulasan_gojek_bersih.csv'")

Proses berhasil! DataFrame yang sudah bersih telah disimpan ke file 'ulasan_gojek_bersih.csv'


Setelah Pembersihan

In [3]:
nama_file = '/content/sample_data/ulasan_gojek_bersih.csv'
df_bersih = pd.read_csv(nama_file)
print(df_bersih.head())

                               reviewId         userName  \
0  c650db79-43ec-413a-9c5f-65101db02ca0  Pengguna Google   
1  acb71a96-9f38-4686-91c8-b6aee9662026  Pengguna Google   
2  cb39a538-ea0e-4de8-a8ce-28f8cb972170  Pengguna Google   
3  720a9ac3-75d2-4bec-8a29-933de466cf8f  Pengguna Google   
4  40e74571-4913-49bb-b75d-60045df0e598  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0  aplikasi ini sangat membantu saya untuk order ...      5              0   
1  Drivernya banyak, mau ambil pas hujan juga, ga...      5              0   
2                    Wow uda 10 tahin aja 30 09 2025      5         

In [4]:
df_bersih.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              11000 non-null  object
 1   userName              11000 non-null  object
 2   userImage             11000 non-null  object
 3   content               11000 non-null  object
 4   score                 11000 non-null  int64 
 5   thumbsUpCount         11000 non-null  int64 
 6   reviewCreatedVersion  8547 non-null   object
 7   at                    11000 non-null  object
 8   replyContent          3035 non-null   object
 9   repliedAt             3035 non-null   object
 10  appVersion            8547 non-null   object
 11  clean_content         10692 non-null  object
dtypes: int64(2), object(10)
memory usage: 1.0+ MB


In [ ]:
'''
# 1. Definisikan fungsi untuk mengubah rating menjadi label sentimen
def tentukan_sentimen(rating):
    if rating <= 2:
        return 'negatif'  # Rating 1 dan 2 dianggap negatif
    elif rating == 3:
        return 'netral'   # Rating 3 dianggap netral
    else: # rating 4 atau 5
        return 'positif'  # Rating 4 dan 5 dianggap positif
'''


In [ ]:
'''# 2. Terapkan fungsi tersebut pada kolom 'score' untuk membuat kolom baru 'sentimen'
df_bersih['sentimen'] = df_bersih['score'].apply(tentukan_sentimen)
'''

In [5]:
print(df_bersih[['score','clean_content']].head())


   score                                      clean_content
0      5  aplikasi bantu order gojek berangkat pulang kerja
1      5  drivernya ambil pas hujan ga kaya grab yg maha...
2      5                                  wow uda tahin aja
3      5                                               good
4      1  kecewa bgt ga segercep grab pake gojek kapok b...


In [6]:
# Ini untuk memastikan data Anda seimbang atau tidak
print("\nDistribusi jumlah data per kelas sentimen:")
print(df_bersih['score'].value_counts())


Distribusi jumlah data per kelas sentimen:
score
5    7118
1    2569
4     549
3     410
2     354
Name: count, dtype: int64


Menentukan X DAN Y

In [7]:
df_final = df_bersih[['clean_content', 'score']]
nama_file_final = 'dataset_final3.csv'
df_final.to_csv(nama_file_final, index=False)


====================